In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import jieba
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import roc_curve, auc, classification_report,accuracy_score,precision_score,f1_score,recall_score,cohen_kappa_score,multilabel_confusion_matrix,make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import metrics
import joblib
from scipy import sparse
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.model_selection import cross_val_score,cross_validate

In [2]:
# load data 
dataframe = pd.read_csv(r'C:\Users\dawns\Desktop\Master thesis\data\Features_level.csv')
#data=dataframe[['CONTENT_NEW','','SCORE']].copy()
df_data=dataframe[['CONTENT_NEW','WORDCUT','SCORE','LEVEL','Tokens','level1_tokens','level2_tokens','level3_tokens','level4_tokens','level5_tokens','level6_tokens','level7_tokens','char_num','high_level','primary_level','Connective_ratio','connective_SCORE','pre','connective_numbers']].copy()
df_data.head()

,CONTENT_NEW,WORDCUT,SCORE,LEVEL,Tokens,level1_tokens,level2_tokens,level3_tokens,level4_tokens,level5_tokens,level6_tokens,level7_tokens,char_num,high_level,primary_level,Connective_ratio,connective_SCORE,pre,connective_numbers
0,记得当年年纪小，家中兄弟姐妹数十多人。我排行倒最第二。父母亲为了养起这头家，日夜往外奔波劳累...,记得 当年 年纪 小 ， 家中 兄弟姐妹 数十 多人 。 我 排行 倒 最 第二 。 父母亲...,90,3,342,0,2,7,4,2,0,3,636,5,13,0.067251,2.984263,记得 当年 年纪 小 家中 兄弟姐妹 数十 多人 我 排行 倒 最 第二 父母亲 为了 养起...,23
1,《一封求职信》×××揭阳县西门外磐东镇肇沟乡广东省李文先生快乐家庭旅游公司负责人广东省李文先...,《 一封 求职信 》 × × × 揭阳县 西 门外 磐 东镇肇沟 乡 广东省 李文 先生 快...,80,3,194,0,0,0,3,0,0,5,403,5,3,0.051546,2.434391,一封 求职信 揭阳县 西 门外 磐 东镇肇沟 乡 广东省 李文 先生 快乐 家庭 旅游 公司...,10
2,我的外婆对我的影响非常大。从小，我父母便将我交给外婆带。他们是上班一族，平日无法照顾我。虽然...,我 的 外婆 对 我 的 影响 非常 大 。 从小 ， 我 父母 便 将 我 交给 外婆 带...,80,3,302,0,4,5,5,3,0,4,570,7,14,0.072848,3.791400,我 的 外婆 对 我 的 影响 非常 大 从小 我 父母 便 将 我 交给 外婆 带 他们 ...,22
3,“安乐死”究竟可不可取？在二十世纪的今天，“安乐死”已不是什么新话题。在不同的国家，已曾有人...,“ 安乐死 ” 究竟 可不 可取 ？ 在 二十世纪 的 今天 ， “ 安乐死 ” 已 不是 ...,95,3,273,1,0,4,3,1,1,8,588,10,8,0.084249,3.982972,安乐死 究竟 可不 可取 在 二十世纪 的 今天 安乐死 已 不是 什么 新 话题 在 不同...,23
4,关于安乐死这个问题，近来引起了广泛的讨论，有些人认为：一个人既既身罹不治之症，与其留在世上饱...,关于 安乐死 这个 问题 ， 近来 引起 了 广泛 的 讨论 ， 有些 人 认为 ： 一个 ...,85,3,186,1,1,5,4,1,0,4,364,5,11,0.096774,2.989940,关于 安乐死 这个 问题 近来 引起 了 广泛 的 讨论 有些 人 认为 一个 人 既 既身...,18


In [3]:
# Generating 'y' for classification as well as regression
y = df_data['SCORE']

In [4]:

# generating x 
pre = df_data['pre']
# Tokenization
def preprocessing_sentence(x):
    words = jieba.cut(str(x).strip())
    return ' '.join(words)

In [5]:
# load tf-idf vector
vector = joblib.load('vector.pkl') 
vector.shape


(3668, 36035)

In [6]:
# 特征合并 feature concatenate 
feature_set1 = df_data[['connective_SCORE',]]
feature_set2 = vector
print(feature_set1.shape)
print(feature_set2.shape)
train_features = sparse.hstack((feature_set2, feature_set1))
print('=============')
print(train_features.shape)

(3668, 1)
(3668, 36035)
(3668, 36036)


In [7]:
# 划分训练集 ，测试集 train_test split
x_train, x_test, y_train, y_test = train_test_split(feature_set2, y, test_size=0.2)
print(x_train.shape)
print(y_train.shape)

(2934, 36035)
(2934,)


In [8]:
#Tf-IDF + LR 
#评估
# （准确率、精确度、召回率、F1值、Kappa系数用于评估分类任务）
## 二分类问题，一般设为两类，正类/反类，或0、1. 所以有tp，fn，fp，tn。 t，f代表是否正确，p，n代表是正类还是反类
## 准确率（accuracy）：所有分类正确的样本数，除以样本总数）A= （tp+tn）/（tp+tn+fp+fn）
## 精确度（precision）精确度也叫查准率，判为正的为p，包括正确tp，和错误fp，其中tp即预测正确的比例 P= tp/tp+fp
## 召回率（recall） ： 查全率，判断正确的且为正的是tp，判断错误且为负（该正却负的），想知道tp占实际正确的比例 r= tp/tp+fn
## f1 越大越好，f1=2*tp/(2*tp+fp+fn)
## kappa 衡量分类精度，其计算基于混淆矩阵，目前用的是二次加权kappa，对错误的惩罚更重，应用更广泛
# 逻辑回归LR
lr = LogisticRegression()
lr.fit(x_train, y_train)
train_score = lr.score(x_train, y_train)
y_pred = lr.predict(x_test)
print("Train Accuracy:", train_score)
print("================================================")
y_pred = lr.predict(x_test)
test_score = accuracy_score(y_test, y_pred)
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)


Train Accuracy: 0.696319018404908
Test Accuracy: 0.24114441416893734
acc_for_each_class:
 [0.         0.         0.         0.33333333 0.20408163 0.125
 0.28981723 0.11363636 0.21311475 0.20689655 0.31914894 0.        ]
qwkappa:
 0.4308102369221837
Weighted precision 0.20141792111569948
Weighted recall 0.24114441416893734
Weighted f1-score 0.19279278045155968
classification report: 
               precision    recall  f1-score   support

          40       0.00      0.00      0.00         4
          45       0.00      0.00      0.00         7
          50       0.00      0.00      0.00        24
          55       0.33      0.08      0.12        39
          60       0.20      0.12      0.15        81
          65       0.12      0.15      0.14        93
          70       0.29      0.70      0.41       159
          75       0.11      0.05      0.07       105
          80       0.21      0.16      0.18        83
          85       0.21      0.09      0.12        68
          90      

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are

In [9]:
# randomforest classfier
rf = ensemble.RandomForestClassifier()
params = {'n_estimators': [50], 'max_depth':[100], 'max_features':[10]}
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)
train_score = rf.score(x_train, y_train)
test_score = accuracy_score(y_test, y_pred)
print("Train Accuracy:", train_score)
print("================================================")
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)



Train Accuracy: 0.9993183367416496
Test Accuracy: 0.25204359673024523
acc_for_each_class:
 [1.         0.         0.         0.         0.275      0.14285714
 0.27348643 0.38461538 0.15789474 0.17647059 0.24390244 0.        ]
qwkappa:
 0.39340743389447097
Weighted precision 0.21997515596250047
Weighted recall 0.25204359673024523
Weighted f1-score 0.18502935311404284
classification report: 
               precision    recall  f1-score   support

          40       1.00      0.25      0.40         4
          45       0.00      0.00      0.00         7
          50       0.00      0.00      0.00        24
          55       0.00      0.00      0.00        39
          60       0.28      0.14      0.18        81
          65       0.14      0.11      0.12        93
          70       0.27      0.82      0.41       159
          75       0.38      0.10      0.15       105
          80       0.16      0.11      0.13        83
          85       0.18      0.04      0.07        68
          9

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are

In [10]:
# support vector machine classfier
model = OneVsRestClassifier(svm.SVC(kernel='rbf',gamma='scale'))#linear
clt = model.fit(x_train,y_train)
y_pred = clt.predict(x_test)
test_score = accuracy_score(y_test, y_pred)
train_score = clt.score(x_train, y_train)
print("Train Accuracy:", train_score)
print("================================================")
print("Test Accuracy:", test_score)
print("================================================")
acc_for_each_class = metrics.precision_score(y_test,y_pred,average=None)
print("acc_for_each_class:\n",acc_for_each_class)
print("================================================")
qwkappa = cohen_kappa_score(y_test,y_pred,weights='quadratic')
print("qwkappa:\n",qwkappa)
print("================================================")
print('Weighted precision', precision_score(y_test, y_pred, average='weighted'))
print('Weighted recall', recall_score(y_test, y_pred, average='weighted'))
print('Weighted f1-score', f1_score(y_test, y_pred, average='weighted'))
print("================================================")
classification_rep = classification_report(y_test,y_pred)
print("classification report: \n",classification_rep)
print("================================================")
confusion_mat = metrics.confusion_matrix(y_test, y_pred)
print(confusion_mat)

Train Accuracy: 0.9972733469665985
Test Accuracy: 0.23841961852861035
acc_for_each_class:
 [1.         0.         0.         0.23076923 0.25       0.13
 0.30407524 0.1754386  0.1509434  0.18367347 0.22535211 0.        ]
qwkappa:
 0.4392514786589893
Weighted precision 0.20309369862536084
Weighted recall 0.23841961852861035
Weighted f1-score 0.2037825517266135
classification report: 
               precision    recall  f1-score   support

          40       1.00      0.50      0.67         4
          45       0.00      0.00      0.00         7
          50       0.00      0.00      0.00        24
          55       0.23      0.08      0.12        39
          60       0.25      0.21      0.23        81
          65       0.13      0.14      0.13        93
          70       0.30      0.61      0.41       159
          75       0.18      0.10      0.12       105
          80       0.15      0.10      0.12        83
          85       0.18      0.13      0.15        68
          90       

D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are

In [11]:
# 交叉验证 cross validation   logistic regression
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(lr, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

{'fit_time': array([6.4655633 , 5.64866948, 5.71621561, 5.73868799, 6.14632344,
       7.2304275 , 5.98521471, 6.03314233, 5.87851954, 6.56491518]), 'score_time': array([0.        , 0.        , 0.01015139, 0.00298691, 0.00398993,
       0.00299311, 0.00299191, 0.        , 0.00395846, 0.00299215]), 'test_accuracy': array([0.24489796, 0.23469388, 0.25510204, 0.2585034 , 0.2662116 ,
       0.27645051, 0.26279863, 0.26962457, 0.23549488, 0.25255973]), 'test_qwkappa': array([0.27166365, 0.33434757, 0.37629707, 0.41281156, 0.40192433,
       0.36943111, 0.38968035, 0.31584907, 0.39973944, 0.34251306])}
Mean accuracy: 0.2556337210652179
Mean qwkappa: 0.36142572107312154


In [12]:
# random forest classification
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(rf, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

{'fit_time': array([6.42405868, 7.4604249 , 7.05094743, 6.86362767, 5.89876533,
       6.31166172, 6.45913434, 6.9657867 , 6.73577881, 7.13683224]), 'score_time': array([0.04188919, 0.03033066, 0.03025126, 0.04041123, 0.03042388,
       0.03956914, 0.03687882, 0.03789926, 0.03494239, 0.03493738]), 'test_accuracy': array([0.25170068, 0.24489796, 0.2585034 , 0.23469388, 0.24914676,
       0.27303754, 0.27303754, 0.24573379, 0.26962457, 0.28327645]), 'test_qwkappa': array([0.31966311, 0.35394889, 0.33389131, 0.4790088 , 0.3835883 ,
       0.35999126, 0.41436771, 0.25690167, 0.45913552, 0.38442735])}
Mean accuracy: 0.2583652573657449
Mean qwkappa: 0.3744923925200214


In [13]:
# support machine classification
scoring = {'accuracy': 'accuracy','qwkappa': make_scorer(cohen_kappa_score,weights='quadratic')}
scores = cross_validate(clt, x_train, y_train, cv=10, scoring=scoring)#10-fold cv
accuracy = scores['test_accuracy']
qwkappa = scores['test_qwkappa']
print(scores)
print('Mean accuracy:',accuracy.mean())
print('Mean qwkappa:',qwkappa.mean())

{'fit_time': array([62.18938112, 62.58284211, 59.71635818, 61.61911249, 62.55177879,
       63.09618592, 57.36302924, 61.68565249, 62.74765944, 67.26218104]), 'score_time': array([6.59657931, 6.17828774, 6.20413661, 6.31342006, 6.15306091,
       6.2917428 , 5.9267292 , 6.65033507, 6.39968586, 7.03215337]), 'test_accuracy': array([0.25510204, 0.25510204, 0.26870748, 0.27210884, 0.25938567,
       0.27303754, 0.27303754, 0.25938567, 0.25938567, 0.2662116 ]), 'test_qwkappa': array([0.3779268 , 0.43122926, 0.41654282, 0.51179283, 0.50310476,
       0.48209184, 0.42860351, 0.36950136, 0.47672165, 0.44499014])}
Mean accuracy: 0.26414640941700906
Mean qwkappa: 0.4442504968122649
